In [35]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, explained_variance_score, mean_absolute_error

In [3]:
preds = {'rxtr': 'reactor', 'burn' : 'burnup', 'cool' : 'cooling', 'enri' : 'enrichment'}
algs = ['knn', 'dtree']
fracs = ['1.0']#['0.1', '0.3', '1.0']
mll_frac = '1.0'
learn29 = {}
base = '/mnt/researchdrive/BOX_INTERNAL/opotowsky/'
sci_path = base + 'scikit/conc_sfco/'
path_end = '_nuc29_ext_test_compare.csv'
for frac in fracs:
    for p, pred in preds.items():
        for alg in algs:
            key = p + frac + '_' + alg
            mid = pred + '_' + alg + '_tset' + frac
            learn29[key] = pd.read_csv(sci_path + mid + path_end, index_col=0)

mll29 = pd.read_csv(base + 'mll/nuc_conc/sfco/Job0_unc0.05/Job0_unc0.05.csv', index_col=0)

## New code only looking at full trainset

In [72]:
cols = ['kNN', 'kNN Std', 'DTree', 'DTree Std', 'MaxLL', 'MaxLL Std']
rows = ['rxtr', 'burn', 'enri']
llparam = { 'rxtr' : ['ReactorType', 'pred_ReactorType'],
            'burn' : ['Burnup', 'pred_Burnup'],
            'enri' : ['Enrichment', 'pred_Enrichment'] }
sfco_errs = pd.DataFrame(index=rows, columns=cols)

In [73]:
#overwrite algs:
algs = ['kNN', 'DTree', 'MaxLL']
for pred in rows:
    for alg in algs:
        key = pred + '1.0'
        std = alg + ' Std'
        if alg == 'kNN':
            key = key + '_knn'
        else:
            key = key + '_dtree'
        if pred == 'rxtr':
            if alg == 'MaxLL':
                sfco_errs.loc[pred, alg] = np.around(accuracy_score(mll29[llparam[pred][0]], mll29[llparam[pred][1]]), decimals=2)
                sfco_errs.loc[pred, std] = np.around(np.std(mll29[llparam[pred][0]] == mll29[llparam[pred][1]]), decimals=2)
            else:
                sfco_errs.loc[pred, alg] = np.around(accuracy_score(learn29[key]['TrueY'], learn29[key][alg]), decimals=2)
                sfco_errs.loc[pred, std] = np.around(np.std(learn29[key]['TrueY'] == learn29[key][alg]), decimals=2)
        elif pred == 'burn':
            gw = 1000.0
            if alg == 'MaxLL':
                sfco_errs.loc[pred, alg] = np.around(mean_absolute_error(mll29[llparam[pred][0]], mll29[llparam[pred][1]])/gw, decimals=2)
                sfco_errs.loc[pred, std] = np.around(np.std(mll29[llparam[pred][0]] - mll29[llparam[pred][1]])/gw, decimals=2)
            else:
                sfco_errs.loc[pred, alg] = np.around(mean_absolute_error(learn29[key]['TrueY'], learn29[key][alg])/gw, decimals=2)
                sfco_errs.loc[pred, std] = np.around(np.std(learn29[key]['TrueY'] - learn29[key][alg])/gw, decimals=2)
        else:
            if alg == 'MaxLL':
                sfco_errs.loc[pred, alg] = np.around(mean_absolute_error(mll29[llparam[pred][0]], mll29[llparam[pred][1]]), decimals=2)
                sfco_errs.loc[pred, std] = np.around(np.std(mll29[llparam[pred][0]] - mll29[llparam[pred][1]]), decimals=2)
            else:
                sfco_errs.loc[pred, alg] = np.around(mean_absolute_error(learn29[key]['TrueY'], learn29[key][alg]), decimals=2)
                sfco_errs.loc[pred, std] = np.around(np.std(learn29[key]['TrueY'] - learn29[key][alg]), decimals=2)

In [74]:
new_idx = {'rxtr' : 'Reactor Type Accuracy',
           'burn' : 'Burnup MAE [GWd/MTU]',
           'enri' : 'Enrichment MAE [%U235]'
          }
sfco_errs.rename(index=new_idx, inplace=True)

In [75]:
sfco_errs

,kNN,kNN Std,DTree,DTree Std,MaxLL,MaxLL Std
Reactor Type Accuracy,0.61,0.49,0.51,0.5,0.72,0.45
Burnup MAE [GWd/MTU],15.17,13.23,14.03,15.32,3.46,5
Enrichment MAE [%U235],1.48,1.18,0.33,0.5,0.48,0.63


In [77]:
import dataframe_image as dfi
dfi.export(sfco_errs, 'sfcoerrs.png')

## Saving code for comparing multiple trainset fracs

In [5]:
Algs = ['kNN', 'DTree']
cols = Algs + ['MaxLL']
cols

['kNN', 'DTree', 'MaxLL']

In [6]:
rxtr29 = pd.DataFrame(columns=cols)
for frac in fracs:
    scores29 = []
    for col in cols:
        if col in Algs:
            key = 'rxtr' + frac
            if 'kNN' in col:
                key = key + '_knn'
                scores29.append(accuracy_score(learn29[key]['TrueY'], learn29[key]['kNN']))
            else:
                key = key + '_dtree'
                scores29.append(accuracy_score(learn29[key]['TrueY'], learn29[key]['DTree']))
        else:
            if frac == mll_frac:
                scores29.append(accuracy_score(mll29['ReactorType'], mll29['pred_ReactorType']))
            else:
                scores29.append(np.nan)
    rxtr29.loc[frac] = scores29
print('Reactor Type, 29 Nuclides: \n')
rxtr29

Reactor Type, 29 Nuclides: 



,kNN,DTree,MaxLL
1.0,0.613861,0.512871,0.718812


In [7]:
llparam = { 'burn' : ['Burnup', 'pred_Burnup'],
            'cool' : ['CoolingTime', 'pred_CoolingTime'],
            'enri' : ['Enrichment', 'pred_Enrichment'] }

In [8]:
def explained_variances(param):
    df29 = pd.DataFrame(columns=cols)
    for frac in fracs:
        scores29 = []
        for col in cols:
            if col in Algs:
                key = param + frac
                if 'kNN' in col:
                    key = key + '_knn'
                    scores29.append(explained_variance_score(learn29[key]['TrueY'], learn29[key]['kNN']))
                else:
                    key = key + '_dtree'
                    scores29.append(explained_variance_score(learn29[key]['TrueY'], learn29[key]['DTree']))
            else:
                if frac == mll_frac:
                    scores29.append(explained_variance_score(mll29[llparam[param][0]], mll29[llparam[param][1]]))
                else:
                    scores29.append(np.nan)
        df29.loc[frac] = scores29
    print(llparam[param][0] + ', ExpVar, 29 nuclides: \n')
    return df29

In [9]:
def mean_abs_error(param):
    df29 = pd.DataFrame(columns=cols)
    for frac in fracs:
        scores29 = []
        for col in cols:
            if col in Algs:
                key = param + frac
                if 'kNN' in col:
                    key = key + '_knn'
                    scores29.append(mean_absolute_error(learn29[key]['TrueY'], learn29[key]['kNN']))
                else:
                    key = key + '_dtree'
                    scores29.append(mean_absolute_error(learn29[key]['TrueY'], learn29[key]['DTree']))
            else:
                if frac == mll_frac:
                    scores29.append(mean_absolute_error(mll29[llparam[param][0]], mll29[llparam[param][1]]))
                else:
                    scores29.append(np.nan)
        df29.loc[frac] = scores29
    print(llparam[param][0] + ', MAE, 29 nuclides: \n')
    return df29

In [10]:
enri29 = mean_abs_error('enri')
enri29

Enrichment, MAE, 29 nuclides: 



,kNN,DTree,MaxLL
1.0,1.477326,0.329612,0.484412


In [11]:
burn29 = mean_abs_error('burn')
burn29

Burnup, MAE, 29 nuclides: 



,kNN,DTree,MaxLL
1.0,15172.076809,14027.794601,3462.108343


In [9]:
enri29 = explained_variances('enri')
enri29

Enrichment, ExpVar, 29 nuclides: 



,kNN,DTree,MaxLL
0.1,-0.567388,0.655351,NaN
0.3,-0.585302,0.709187,NaN
1.0,-0.561107,0.721632,0.557847


In [11]:
burn29 = explained_variances('burn')
burn29

Burnup, ExpVar, 29 nuclides: 



,kNN,DTree,MaxLL
0.1,0.424760,0.195675,NaN
0.3,0.425014,0.186354,NaN
1.0,0.435253,0.242814,0.919398
